In [7]:
import math
from collections import namedtuple
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from copy import deepcopy
from tqdm import tqdm
from tqdm.notebook import tqdm
import random

import numba as nb
# from numba import jit, vectorize, float64

In [8]:
with open('./data/tsp_51_1', 'r') as input_data_file:
    input_data = input_data_file.read()
    
lines = input_data.split('\n')
nodeCount = int(lines[0])

points = []
for i in range(1, nodeCount+1):
    line = lines[i]
    parts = line.split()
    # points.append(Point(float(parts[0]), float(parts[1])))
    points.append((float(parts[0]), float(parts[1])))
points = np.array(points)

In [33]:
@nb.jit(nopython=True)
def length_distance(single_point, all_points):
    return np.sqrt((all_points[:, 0]-single_point[0])**2 + (all_points[:, 1]-single_point[1])**2)


@nb.jit(nopython=True)
def calculate_travel(graph):
    n = len(graph)
    total_distance = 0.0

    for i in range(n - 1):
        total_distance += DISTANCE_MATRIX[graph[i], graph[i + 1]]

    # Add the distance to return to the starting point
    total_distance += DISTANCE_MATRIX[graph[-1], graph[0]]

    return total_distance



In [41]:
def init_path(node_count):
    ## Keep picking the next nearest neighbor to get the path.
    pick_next = 0
    exploring_path = np.array([], dtype=int)
    exploring_path = np.append(exploring_path, pick_next)
    
    while exploring_path.size < nodeCount:
        
        neighbor_idx = np.argsort(DISTANCE_MATRIX[pick_next])
        
        mask = np.isin(neighbor_idx, exploring_path)
        neighbor_idx = neighbor_idx[~mask]
    
        pick_next = neighbor_idx[0]
        
        exploring_path = np.append(exploring_path, pick_next)

    total_distance = calculate_travel(exploring_path)

    return exploring_path, total_distance

In [42]:
DISTANCE_MATRIX = np.zeros((nodeCount, nodeCount))
for i in tqdm(range(len(points))):
    DISTANCE_MATRIX[i] = length_distance(points[i], points)

  0%|          | 0/51 [00:00<?, ?it/s]

In [43]:
class Penalty:
    def __init__(self, node_count):
        self.penalty = [[0] * i for i in range(1, node_count + 1)]

    def __getitem__(self, key):
        j, i = sorted(key)
        return self.penalty[i][j]

    def __setitem__(self, key, value):
        j, i = sorted(key)
        self.penalty[i][j] = value

In [44]:
class Activate:
    def __init__(self, size):
        self.flag = [1] * size
        self.count_ = size

    def set_1(self, i):
        if self.flag[i] == 0:
            self.count_ += 1
        self.flag[i] = 1

    def set_0(self, i):
        if self.flag[i] == 1:
            self.count_ -= 1
        self.flag[i] = 0

    def get(self, i):
        return self.flag[i]

    def size(self):
        return len(self.flag)

In [163]:
do_2opt(3, 4, current_path)

array([ 0, 33,  5, 28,  2, 45,  9, 10,  3, 46, 24, 41, 34, 23, 35,  4,  8,
       12, 36,  6, 26, 47, 27,  1, 25, 20, 37, 21, 29, 43, 39, 50, 38, 15,
       44, 16, 18, 40, 19,  7, 13, 30, 11, 42, 31, 22, 48, 32, 17, 49, 14],
      dtype=int64)

In [198]:
# @nb.jit(nopython=True)
def do_2opt(point_idx1, point_idx2, path, new_distance, augmented_distance):
    if point_idx2 > point_idx1:
        slice_start = point_idx1
        slice_end = point_idx2
    else:
        slice_start = point_idx2
        slice_end = point_idx1

    slice_path = path[slice_start:slice_end+1]
    slice_path = np.flip(slice_path)

    new_path = np.hstack((path[:slice_start], slice_path, path[slice_end + 1:]))

    point1 = current_path[pointidx1-1]
    point2 = current_path[pointidx1]
    point3 = current_path[pointidx2]
    point4 = current_path[pointidx2+1] if pointidx2+1 < nodeCount else current_path[0]

    d12 = DISTANCE_MATRIX[point1][point2]
    p12 = PENALTY[point1, point2]
    
    d34 = DISTANCE_MATRIX[point3][point4]
    d13 = DISTANCE_MATRIX[point1][point3]
    d24 = DISTANCE_MATRIX[point2][point4]
    p34 = PENALTY[point3, point4]
    p13 = PENALTY[point1, point3]
    p24 = PENALTY[point2, point4]

    net_distance = d13 + d24 - d12 - d34
    augmented_distance_delta =  net_distance + lambda_factor * (p13 + p24 - p12 - p34)

    new_distance += net_distance
    augmented_distance += augmented_distance_delta 
    
    return new_path, new_distance, augmented_distance


def pick_next_edge(pointidx1, current_path):

    point1 = current_path[pointidx1-1]
    point2 = current_path[pointidx1]
    
    d12 = DISTANCE_MATRIX[point1][point2]
    p12 = PENALTY[point1, point2]
    
    
    # Repeatedly do 2-opt for all the neighbors
    next_probable_edge = {}
    max_distance_delta = float('inf')
    
    node = current_path[pointidx1]
    sorted_neighbor_points = np.argsort(DISTANCE_MATRIX[node])[1:SEARCH_AREA]
    
    for neighbor_idx in sorted_neighbor_points:
       
        point_idx2 = np.where(current_path==neighbor_idx)[0][0]
        
        point3 = current_path[pointidx2]
        point4 = current_path[pointidx2+1] if pointidx2+1 < nodeCount else current_path[0]
    
    
        d34 = DISTANCE_MATRIX[point3][point4]
        d13 = DISTANCE_MATRIX[point1][point3]
        d24 = DISTANCE_MATRIX[point2][point4]
        
        p34 = PENALTY[point3, point4]
        p13 = PENALTY[point1, point3]
        p24 = PENALTY[point2, point4]
    
        augmented_distance_delta = (d13 + d24 - d12 - d34) + lambda_factor * (p13 + p24 - p12 - p34)
    
        next_probable_edge[point_idx2] = augmented_distance_delta

    # filter candidates with max value
    max_distance_delta = max(next_probable_edge.values())
    print(max_distance_delta)
    viable_candidates = [key for key, value in next_probable_edge.items() if value == max_distance_delta]

    # Pick random viable candidate
    point_idx2 = random.choice(viable_candidates)
    point3 = current_path[pointidx2]
    point4 = current_path[pointidx2+1] if pointidx2+1 < nodeCount else current_path[0]
    
    return (point_idx1, point_idx2), (point1, point2, point3, point4)

In [199]:
step_limit = 1000000
SEARCH_AREA = nodeCount//5
PENALTY = Penalty(nodeCount)
activate = Activate(nodeCount)

current_path, current_distance = init_path(nodeCount)

alpha = 0.1
lambda_factor = alpha * current_distance/nodeCount

In [200]:
current_distance

566.9434953326025

In [201]:
do_2opt(3, 4, current_path, current_distance, current_distance)

(array([ 0, 33,  5, 28,  2, 45,  9, 10,  3, 46, 24, 41, 34, 23, 35,  4,  8,
        12, 36,  6, 26, 47, 27,  1, 25, 20, 37, 21, 29, 43, 39, 50, 38, 15,
        44, 16, 18, 40, 19,  7, 13, 30, 11, 42, 31, 22, 48, 32, 17, 49, 14],
       dtype=int64),
 585.2633956142943,
 585.2633956142943)

In [ ]:
def update_penalty(self):
        self.utility = np.zeros((self.n, self.n), dtype=np.float32)
        for i in range(self.n):
            for j in range(self.n):
                dist = self.dist[i][self.solution[i]]
                flow = self.flow[i][j]
                c    = dist * flow
                self.utility[i][self.solution[i]] = c / (1 + self.penalty[i][self.solution[i]])
        maximized = self.utility.max()
        for i in range(self.n):
            if self.utility[i][self.solution[i]] == maximized:
                self.penalty[i][self.solution[i]] += 1